In [ ]:
import cobra
import pandas as pd
import os

pd.set_option('display.max_rows', None)


s12 = cobra.io.read_sbml_model(os.getcwd()+'\\input\\iSH1474.xml')
kt2440 = cobra.io.read_sbml_model(os.getcwd()+'\\input\\iJN1463.xml')

In [3]:
pm = pd.read_excel(os.getcwd()+'\\input\\PM_result.xlsx')
pm

,PM test metabolites,metabolites id,s12 growth
0,A02 (L-Arabinose),arab__L,-
1,A03 (N-Acetyl-D-Glucosamine),acgam,-
2,A04 (D-Saccharic Acid),glcr,+
3,A05 (Succinic Acid),succ,+
4,A06 (D-Galactose),gal,-
5,A07 (L-Aspartic Acid),asp__L,+
6,A08 (L-Proline),pro__L,+
7,A09 (D-Alanine),ala__D,+
8,A10 (D-Trehalose),tre,-
9,A11 (D-Mannose),man,+


In [3]:
def check (x, y):
    if x == '+':
        if y == '+':
            ans = 'TP'
        else:
            ans = 'FN'
    else:
        if y == '+':
            ans = 'FP'
        else:
            ans = 'TN'
            
    return ans


def growth_simulation (model):

    metabolites = pm['metabolites id'].values.tolist()
    exp_result = pm['s12 growth'].values.tolist()
    
    simulation_result = []
    min_growth = model.slim_optimize()*0.05

    for i in range(len(metabolites)):
        m = metabolites[i]
        x = exp_result[i]
        
        if m == 'x':
            y = '-'
        else:
            with model:
                m_id = 'EX_'+m+'_e'
            
                model.reactions.get_by_id('EX_glc__D_e').bounds = (0,0)
                
                try:
                    model.reactions.get_by_id(m_id).lower_bound = -6
            
                    simulation_growth = model.slim_optimize()
            
                    if simulation_growth > min_growth:
                        y = '+'
                    else:
                        y = '-'
                    
                except:
                    y = '-'
                    
                    
        simulation_result.append(check (x, y))
                    
    return simulation_result

In [13]:
for gem in ['s12','kt2440']:
    model = globals()[gem]
    simulation_result = growth_simulation(model)
    pm[gem+' simulation result'] = simulation_result

In [23]:
data = {'TP':[],'TN':[],'FP':[],'FN':[]}

data['TP'].append(pm['s12 simulation result'].to_list().count('TP'))
data['TN'].append(pm['s12 simulation result'].to_list().count('TN'))
data['FP'].append(pm['s12 simulation result'].to_list().count('FP'))
data['FN'].append(pm['s12 simulation result'].to_list().count('FN'))

data['TP'].append(pm['kt2440 simulation result'].to_list().count('TP'))
data['TN'].append(pm['kt2440 simulation result'].to_list().count('TN'))
data['FP'].append(pm['kt2440 simulation result'].to_list().count('FP'))
data['FN'].append(pm['kt2440 simulation result'].to_list().count('FN'))

display(pd.DataFrame(data,index=['s12','kt2440']))
print('iSH1474 accuracy:',(pm[['s12 simulation result']].value_counts().TP + pm[['s12 simulation result']].value_counts().TN)/203 * 100)
print('iJN1462 accuracy:',(pm[['kt2440 simulation result']].value_counts().TP + pm[['kt2440 simulation result']].value_counts().TN)/203 * 100)

,TP,TN,FP,FN
s12,52,142,0,9
kt2440,46,130,12,15


iSH1474 accuracy: 95.56650246305419
iJN1462 accuracy: 86.69950738916256
